<a href="https://colab.research.google.com/github/Serhii-Kravchenko-2022/Home_work_Data_Science8/blob/main/HwM5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Кравченко Сергій
Модуль 5

# Алгоритми навчання з вчителем. 
Алгоритми SVM та випадковий ліс з бібліотеки scikit-learn

Завантажуємо датасет

In [4]:
import os
import pandas as pd

from google.colab import drive


drive.mount('/DataScience8')

data_folder = '/DataScience8/MyDrive/DataScience8/data'

all_data = pd.DataFrame()

for folder_name in os.listdir(data_folder):
    activity_path = os.path.join(data_folder, folder_name)
    if not os.path.isdir(activity_path):
        continue
    
    for file_name in os.listdir(activity_path):
        file_path = os.path.join(activity_path, file_name)
        if not file_name.endswith('.csv'):
            continue
        
        df = pd.read_csv(file_path)
        
        df['activity'] = folder_name
        
        all_data = pd.concat([all_data, df], ignore_index=True)

all_data = all_data.sample(frac=1).reset_index(drop=True)
all_data.head()



Drive already mounted at /DataScience8; to attempt to forcibly remount, call drive.mount("/DataScience8", force_remount=True).


,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,-12.042834,-12.205640,1.326388,walking
1,-5.702988,-12.052411,-1.130063,walking
2,-5.631162,-2.111686,-1.728614,running
3,-5.329493,-2.485181,-5.755661,running
4,-6.799533,6.502652,1.091756,running


In [12]:
X = all_data.drop('activity', axis=1)
y = all_data['activity']

print('X\n', X.head())
print('y\n', y.head())
print('all_data\n', all_data.shape)

X
    accelerometer_X  accelerometer_Y  accelerometer_Z
0       -12.042834       -12.205640         1.326388
1        -5.702988       -12.052411        -1.130063
2        -5.631162        -2.111686        -1.728614
3        -5.329493        -2.485181        -5.755661
4        -6.799533         6.502652         1.091756
y
 0    walking
1    walking
2    running
3    running
4    running
Name: activity, dtype: object
all_data
 (194160, 4)


Ділимо дані на тренувальні та тестові:

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print('X_train\n', X_train.shape)
print('X_test\n', X_test.shape)
print('y_train\n', y_train.shape)
print('y_test\n', y_test.shape)

X_train
 (135912, 3)
X_test
 (58248, 3)
y_train
 (135912,)
y_test
 (58248,)


Часові ознаки

In [13]:
X_train['time_mean'] = X_train.mean(axis=1)
X_test['time_mean'] = X_test.mean(axis=1)

print('X_train\n', X_train.head())
print('X_test\n', X_test.head())

X_train
         accelerometer_X  accelerometer_Y  accelerometer_Z  time_mean
167747       -11.894394         1.580173        -9.227253  -6.513825
57222          0.033519        10.917560         3.663129   4.871403
118894        -3.639186         0.852336        -3.667917  -2.151589
133829         0.244209        -0.095768         9.753978   3.300806
93898         -0.105345         5.770026         7.613562   4.426081
X_test
         accelerometer_X  accelerometer_Y  accelerometer_Z  time_mean
96885          1.187524        13.072341        -1.953669   4.102065
179740         3.126828       -12.895170         3.840300  -1.976014
58127          2.226608        -8.130708        -4.620809  -3.508303
58714          0.076614       -17.142485        -3.428497  -6.831456
61559         -0.076614         0.244209         9.734824   3.300806


Моделі класіфікації

In [14]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

svm_model = SVC()
rf_model = RandomForestClassifier()

Тренуємо моделі

In [15]:
svm_model.fit(X_train, y_train)

SVC()

In [16]:
rf_model.fit(X_train, y_train)

RandomForestClassifier()

Прогнозування

In [17]:
svm_pred = svm_model.predict(X_test)

In [18]:
rf_pred = rf_model.predict(X_test)

Оцінка результатів

In [19]:
from sklearn.metrics import accuracy_score

svm_accuracy = accuracy_score(y_test, svm_pred)
rf_accuracy = accuracy_score(y_test, rf_pred)

print("Accuracy (SVM):", svm_accuracy)
print("Accuracy (Random Forest):", rf_accuracy)

Accuracy (SVM): 0.8951036945474523
Accuracy (Random Forest): 0.9995364647713226


In [20]:
from sklearn.metrics import classification_report

svm_report = classification_report(y_test, svm_pred)
print("SVM Results:")
print(svm_report)

rf_report = classification_report(y_test, rf_pred)
print("Random Forest Results:")
print(rf_report)

SVM Results:
              precision    recall  f1-score   support

        idle       0.96      0.99      0.97      9267
     running       0.93      0.90      0.92     30999
      stairs       1.00      0.00      0.01      1419
     walking       0.80      0.91      0.85     16563

    accuracy                           0.90     58248
   macro avg       0.92      0.70      0.69     58248
weighted avg       0.90      0.90      0.89     58248

Random Forest Results:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      9267
     running       1.00      1.00      1.00     30999
      stairs       1.00      0.99      1.00      1419
     walking       1.00      1.00      1.00     16563

    accuracy                           1.00     58248
   macro avg       1.00      1.00      1.00     58248
weighted avg       1.00      1.00      1.00     58248



Висновок: точність (accuracy) алгоритму Random Forest вища. Значно швидше виконує тренування (21сек. проти 5 хв.) та прогнозування (1 сек. проти 2 хв.).